In [1]:
import pandas as pd
import numpy as np

In [2]:
map_df = pd.read_csv('Data/mapping.csv')
task_1 = pd.read_csv('Data/task1.csv')
task_2 = pd.read_csv('Data/task2.csv')
task_3 = pd.read_csv('Data/task3.csv')

In [3]:
merge2_df = task_2.replace(
    'other irrelevant ', '24 OTHERS(IRRELEVANT)').replace(
    'other_relevant ', '23 OTHERS(RELEVANT)').replace(
    'research related', '23 OTHERS(RELEVANT)').replace(
    'other irrelevant', '24 OTHERS(IRRELEVANT)').replace(
    'other relevant', '23 OTHERS(RELEVANT)').replace(
    'other_relaevant', '23 OTHERS(RELEVANT)')
merge2_df = merge2_df.rename(columns={'Unnamed: 1': 'cate_1',
                                      'Unnamed: 2': 'cate_2'})

In [4]:
map_df = map_df[['Australian standard']]
map_df['tmp'] = map_df['Australian standard'].str.split('DIVISION ')
map_df = map_df.explode('tmp')
re_rule = '(\d+)'
target_df = map_df[map_df['tmp'].notna()]
target_df['mark'] = target_df['tmp'].str.extract(re_rule)
target_df = target_df[target_df['mark'].notna()][['tmp', 'mark']].drop_duplicates('mark')
target_df['mark'] = target_df['mark'].astype(int).astype(str)
merge_df = task_3.merge(target_df, left_on='key_1', right_on='mark', how='left').merge(
    target_df, left_on='key_2', right_on='mark', how='left').merge(
    target_df, left_on='key_3', right_on='mark', how='left')

merge_df = merge_df[['Tags', 'key_1', 'key_2', 'key_3', 'tmp_x', 'tmp_y', 'tmp']].rename(columns={'tmp_x': 'cate_1', 'tmp_y': 'cate_2', 'tmp': 'cate_3'})
merge_df['cate_1'] = merge_df['cate_1'].fillna(merge_df['key_1'])
merge_df['cate_2'] = merge_df['cate_2'].fillna(merge_df['key_2'])
merge_df['cate_3'] = merge_df['cate_3'].fillna(merge_df['key_3'])
merge_df = merge_df.drop(['key_1','key_2','key_3'], axis=1)
merge3_df = merge_df.replace(
    'other_irrelevant ', '24 OTHERS(IRRELEVANT)').replace(
    'other_relevant ', '23 OTHERS(RELEVANT)').replace(
    'other_irrelevant', '24 OTHERS(IRRELEVANT)').replace(
    'other_relevant', '23 OTHERS(RELEVANT)').replace(
    'other_relaevant', '23 OTHERS(RELEVANT)')

C:\Users\24966\AppData\Local\Temp/ipykernel_94748/2031961785.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df['mark'] = target_df['tmp'].str.extract(re_rule)


In [5]:
merge1_df = task_1.rename(columns={'word': 'Tags', 
                                   'cate1': 'cate_1',
                                   'cate2': 'cate_2',
                                   'cate3': 'cate_3'}).replace(
    'other irrelevant ', '24 OTHERS(IRRELEVANT)').replace(
    'other_relevant ', '23 OTHERS(RELEVANT)').replace(
    'research related', '23 OTHERS(RELEVANT)').replace(
    'other irrelevant', '24 OTHERS(IRRELEVANT)').replace(
    'other relevant', '23 OTHERS(RELEVANT)').replace(
    'other_relaevant', '23 OTHERS(RELEVANT)')

In [6]:
merge_df = pd.concat([merge1_df, merge2_df, merge3_df])

In [7]:
merge_df['Tags'] = merge_df['Tags'].str.lower()

In [17]:
dup_df = merge_df[merge_df.duplicated('Tags',)]
same_list = []
for t in dup_df['Tags'].unique():
    same_list.append(merge_df[merge_df['Tags'] == t])

In [18]:
pd.concat(same_list).to_csv('issue_df.csv', index=0)

In [10]:
def reformat(row):
    for i in ['cate_1', 'cate_2', 'cate_3']:
        if type(row[i]) == type('a'):
            if ',' == row[i][-1]:
                row[i] = row[i][:-1]
            if 'DIVISION' not in row[i]:
                row[i] = 'DIVISION '+row[i]
    return row

In [11]:
merge_df= merge_df.apply(lambda x: reformat(x), axis=1)

In [12]:
merge_df = merge_df.replace(
    'DIVISION 15 COMMERCE, MANAGEMENT, TOURISM', 'DIVISION 15 COMMERCE, MANAGEMENT, TOURISM AND SERVICES').replace(
    'DIVISION 15 COMMERCE,MANAGEMENT,TOURISM AND SERVICES', 'DIVISION 15 COMMERCE, MANAGEMENT, TOURISM AND SERVICES').replace(
    'DIVISION 18 LAW AND LEGAL', 'DIVISION 18 LAW AND LEGAL STUDIES COGNITIVE SCIENCES').replace(
    'DIVISION 18 LAW AND LEGAL STUDIES', 'DIVISION 18 LAW AND LEGAL STUDIES COGNITIVE SCIENCES').replace(
    'DIVISION 19 STUDIES IN CREATIVE ARTS AND WRITIN', 'DIVISION 19 STUDIES IN CREATIVE ARTS AND WRITING').replace(
    'DIVISION 20 LANGUAGE,COMMUNICATION AND CULTURE', 'DIVISION 20 LANGUAGE, COMMUNICATION AND CULTURE').replace(
    'DIVISION 20 LANGUAGE COMMUNICATION AND CULTURE', 'DIVISION 20 LANGUAGE, COMMUNICATION AND CULTURE')

In [23]:
tmp_df = merge_df.melt(id_vars='Tags')[['Tags', 'value']]

In [28]:
tmp_df.dropna().drop_duplicates().to_csv('tag_relation_df.csv', index=0)